In [1]:
import pickle
import numpy
from PIL import Image
import IPython.display

import tracking.Tester
import tracking.RecurrentTracker
import tracking.TheanoGruRnn
import tracking.VideoSequence as vs
import tracking.VisualAttention

/home/fmpaezri/.local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
cd ../tracking

/home/fmpaezri/repos/localization-agent/tracking


In [3]:
with open('/home/fmpaezri/repos/localization-agent/model.pkl0', 'r') as modelFile:
    %time rnn = pickle.load(modelFile)

CPU times: user 8.99 s, sys: 332 ms, total: 9.32 s
Wall time: 9.55 s


In [4]:
tracker = tracking.RecurrentTracker.RecurrentTracker(None, rnn)

In [5]:
tester = tracking.Tester.Tester(tracker)

In [49]:
%time data, labels = tester.loadImageSequence('/home/fmpaezri/datasets/vot-challenge/sequences2014/basketball/', newShape=(rnn.imgSize,rnn.imgSize))

CPU times: user 3.33 s, sys: 52 ms, total: 3.38 s
Wall time: 3.88 s


In [50]:
#Correcting box ccords as expected by IoU
labelsCopy = labels.copy()
labelsCopy[:,0] = labels[:,0::2].min(axis=1) 
labelsCopy[:,1] = labels[:,1::2].min(axis=1)
labelsCopy[:,2] = labels[:,0::2].max(axis=1) 
labelsCopy[:,3] = labels[:,1::2].max(axis=1)
labels = labelsCopy

In [51]:
data = data[numpy.newaxis, ...].astype(numpy.float32)

In [52]:
labels = labels[numpy.newaxis,...]

In [53]:
print data.shape, labels.shape

(1, 725, 100, 100, 3) (1, 725, 4)


In [54]:
%time iou, predLabels = tester.test(data, labels, None, rnn.batchSize, rnn.imgSize, False, labels.shape[1], 4, '/tmp')

CPU times: user 35.8 s, sys: 2.61 s, total: 38.4 s
Wall time: 21.3 s


In [56]:
iou.mean()

0.033065888047635904

In [57]:
seq = vs.VideoSequence((Image.fromarray(frame.astype(numpy.uint8)) for frame in data[0]))

In [58]:
seq.addBoxes(labels[0], 'green')
seq.addBoxes((predLabels[0]), 'red')

In [59]:
outputName = '/tmp/testSequence1.mp4'
seq.exportToVideo(30, outputName)

In [60]:
IPython.display.HTML(vs.displayHTML(outputName))